In [1]:
from theano.sandbox import cuda

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GT 820M (CNMeM is enabled with initial size: 80.0% of memory, cuDNN not available)


In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.
C:\Users\Shikhar\Anaconda2\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
#path = "data/ml-20m/"
path = "data/ml-100k/"
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)
batch_size=64

In [4]:
ratings = pd.read_csv(path+'ratings.csv')
len(ratings)

100000

In [5]:
users = ratings.userId.unique()
movies = ratings.movieId.unique()

In [6]:
userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}

In [7]:
ratings.movieId = ratings.movieId.apply(lambda x: movieid2idx[x])
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])

In [8]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()
n_users, n_movies

(943, 1682)

In [9]:
n_factors = 50

In [10]:
np.random.seed = 42

In [24]:
msk = np.random.rand(len(ratings)) < 0.75
trn = ratings[msk]
val = ratings[~msk]


In [25]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg))(inp)

In [26]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-4)
movie_in, m = embedding_input('movie_in', n_movies, n_factors, 1e-4)

In [45]:

x = merge([u, m], mode='concat')
x = Flatten()(x)
x = Dense(100, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(5,activation='relu')(x)
x = Dense(1)(x)
nn = Model([user_in, movie_in], x)
nn.compile(Adam(0.001), loss='mse')

In [47]:
nn.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=4, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 75148 samples, validate on 24852 samples
Epoch 1/4
75148/75148 [==============================] - 18s - loss: 1.0092 - val_loss: 0.9174
Epoch 2/4
75148/75148 [==============================] - 18s - loss: 0.9354 - val_loss: 0.9080
Epoch 3/4
75148/75148 [==============================] - 18s - loss: 0.8876 - val_loss: 0.9038
Epoch 4/4
75148/75148 [==============================] - 18s - loss: 0.8622 - val_loss: 0.9000


In [48]:
nn.save_weights(model_path+'neural_5epoch_8622_9000')
nn.load_weights(model_path+'neural_5epoch_8622_9000')

In [49]:
nn.predict([np.array([1]), np.array([31])])


array([[3.7652]], dtype=float32)